In [80]:
import numpy as np
import pandas as pd

In [103]:
# import data
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/analysis.p')
subdf = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/subanalysis.p')
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [104]:
def getcount(decision, direction):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [105]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [lnin, expin, strexpin, plwcin]
outcounts = [lnout, expout, strexpout, plwcout]
unkcounts = [lnunk, expunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts

[1395, 1069, 1292, 1702]
[479, 2333, 2016, 0]
[2514, 762, 1016, 2759]


In [108]:
df = df.query("alpha != ''")

In [109]:
doms = np.unique(df.Domain)
# number of graphs and mean alpha value in each domain
counts = np.zeros_like(doms)
means = np.zeros_like(doms)
counts_plaus = np.zeros_like(doms)
means_plaus = np.zeros_like(doms)
counts_plaustail = np.zeros_like(doms)
means_plaustail = np.zeros_like(doms)
for i,dom in enumerate(doms):
    query = "Domain == '%s'" %dom
    subdf = df.query(query)
    counts[i] = len(subdf)
    means[i] = format(np.mean(subdf.alpha), '.2f')
    subdf_plaus = subdf.query("ppl>0.1")
    counts_plaus[i] = len(subdf_plaus)
    means_plaus[i] = format(np.mean(subdf_plaus.alpha), '.2f')
    subdf_plaustail = subdf_plaus.query("ntail>=50")
    counts_plaustail[i] = len(subdf_plaustail)
    means_plaustail[i] = format(np.mean(subdf_plaustail.alpha), '.2f')

In [110]:
print doms
print counts_plaustail
print means_plaustail

['Biological' 'Economic' 'Informational' 'Social' 'Technological'
 'Transportation']
[125 0 13 239 1316 1]
['2.70' 'nan' '2.69' '4.19' '2.19' '4.82']


In [111]:
def test_strong(rows):
    SA = False # strong alone
    S1 = False # strong 1
    S2 = False # strong 2
    n = len(rows)
    strong = 0
    strong_alone = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha < 3 and row.alpha > 2:
            strong_alone += 1
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
                strong += 1
    if strong_alone >= 9.*n/10:
        SA = True
    if strong >= n/2.:
        S2 = True
    if SA == True and strong >= 95.*n/100:
        S1 = True
    return (S1, S2, SA)
            
def test_weak(rows):
    W = False 
    West = False 
    n = len(rows)
    weak = 0
    weakest = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
    if weak >= n/2.:
        W = True
    if weakest >= n/2.:
        West = True
    return (W, West)
          

In [112]:
# test diff hypotheses
unique_datasets = np.unique(df.fp_gml)
hyps = pd.DataFrame(columns = ['Strong1', "Strong2", "Strong_alone", "Weak", "Weakest", "Domain", "Subdomain", "median_alpha"], index=unique_datasets )
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    [S1, S2, SA] = test_strong(rows)
    hyps.loc[dataset]['Strong1'] = S1
    hyps.loc[dataset]['Strong2'] = S2
    hyps.loc[dataset]['Strong_alone'] = SA
    [weak,weakest] = test_weak(rows)
    hyps.loc[dataset]['Weak'] = weak
    hyps.loc[dataset]['Weakest'] = weakest
    hyps.loc[dataset]['Domain'] = rows.Domain[0]
    hyps.loc[dataset]['Subdomain'] = rows.Subdomain[0]
    hyps.loc[dataset]['median_alpha'] = np.median(rows.alpha)

In [113]:
n = float(len(hyps))
Strong1 = np.sum(hyps.Strong1)
Strong2 = np.sum(hyps.Strong2)
Strong_alone = np.sum(hyps.Strong_alone)
Weak = np.sum(hyps.Weak)
Weakest = np.sum(hyps.Weakest)
print "Strong1 = %s" %(Strong1/n)
print "Strong2 = %s" %(Strong2/n)
print "Strong_alone = %s" %(Strong_alone/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Number of Datsets = %s" %int(n)

Strong1 = 0.402777777778
Strong2 = 0.448717948718
Strong_alone = 0.405982905983
Weak = 0.522435897436
Weakest = 0.622863247863
Number of Datsets = 1872


In [114]:
hyps.to_csv('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/hyps.csv')

In [50]:
dataset = unique_datasets[0]
query = "fp_gml == '%s'" %dataset
rows = df.query(query)
np.median(rows.alpha)

5.3500000000000032

In [63]:
len(hyps.query("Domain=='Biological'"))

500

In [120]:
subhyps = hyps.query("Domain=='Social'")
n = float(len(subhyps))
Strong1 = np.sum(subhyps.Strong1)
Strong2 = np.sum(subhyps.Strong2)
Strong_alone = np.sum(subhyps.Strong_alone)
Weak = np.sum(subhyps.Weak)
Weakest = np.sum(subhyps.Weakest)
print "Strong1 = %s" %(Strong1/n)
print "Strong2 = %s" %(Strong2/n)
print "Strong_alone = %s" %(Strong_alone/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Number of Datsets = %s" %int(n)

Strong1 = 0.0278551532033
Strong2 = 0.091922005571
Strong_alone = 0.0278551532033
Weak = 0.345403899721
Weakest = 0.688022284123
Number of Datsets = 359
